# Formal Logic with Python

## Propositional Formulas

A *valid* or *well-formed* propositional formula is a string which satisfies at least one of the following syntactic rules:

1. It is a variable name. A variable name is any letter in ‘p’. . . ‘z’, optionally followed by a sequence of digits. For
example, ‘p’, ‘y12’, or ‘z035’.
2. It is a constant of the form 'T' or 'F'.
4. It is the string '~f', where f is a valid propositional formula.
5. It is the string (a&b), where a and b are valid propositional formulas.
6. It is the string (a|b), where a and b are valid propositional formulas.
7. It is the string (a->b), where a and b are valid propositional formulas.

We can view the above rules as string parsing rules and implement them in Python:

In [1]:
def is_variable(string: str) -> bool:
    return string[0] >= 'p' and string[0] <= 'z' and \
           (len(string) == 1 or string[1:].isdecimal())


def is_constant(string: str) -> bool:
    return string == 'T' or string == 'F'


def is_unary_operator(string: str) -> bool:
    return string == '~'


def is_binary_operator(string: str) -> bool:
    return string == '&' or string == '|' or string == '->'


print('p is a variable:', is_variable('p'))
print('a is a variable:', is_variable('a'))
print('z035 is a variable:', is_variable('z035'))

print('\nT is a constant:', is_constant('T'))
print('F is a constant:', is_constant('F'))
print('z is a constant:', is_constant('z'))

print('\n~ is a unary operator:', is_unary_operator('~'))
print('& is a unary operator:', is_unary_operator('&'))

print('\n& is a binary operator:', is_binary_operator('&'))
print('| is a binary operator:', is_binary_operator('|'))
print('~ is a binary operator:', is_binary_operator('~'))

p is a variable: True
a is a variable: False
z035 is a variable: True

T is a constant: True
F is a constant: True
z is a constant: False

~ is a unary operator: True
& is a unary operator: False

& is a binary operator: True
| is a binary operator: True
~ is a binary operator: False


We can now start implementing a class to store an arbitrarily complex propositional formula as a parse tree. The tree conforms to the following rules:

1. The leaves are proposition symbols.
2. All non-leaves are logical operators.
3. Negation has one child.
4. Binary operators have two children.

If a variable or a constant is passed in as the root, then no children can be passed in. If a unary operator is passed in as the root, then it can only have one child. Two children are only allowed if a binary operator is passed in as the root:

In [2]:
from typing import Self


class PropositionalFormula:
    """
        Attributes:
            root: a constant, variable name, or operator
            first: the first operand of the root, if the root is a unary or binary operator.
            second:the second operand of the root, if the root is a binary operator.
    """
    def __init__(self, root: str, 
                 first: Self = None,
                 second: Self = None
                ):
        if is_variable(root) or is_constant(root):
            assert first is None and second is None
            self.root = root
        elif is_unary_operator(root):
            assert first is not None and second is None
            self.root, self.first = root, first
        else:
            assert is_binary_operator(root)
            assert first is not None and second is not None
            self.root, self.first, self.second = root, first, second

For example, we can represent the propositiona formula '~(p&q)' as follows:

In [3]:
p = PropositionalFormula('p')
q = PropositionalFormula('q')
conjunction = PropositionalFormula('&', p, q)
negation = '~'

propositional_formula = PropositionalFormula('~', conjunction)

print('root of propositional_formula:', propositional_formula.root)
print('first child of propositional formula:', propositional_formula.first.root)
print('first child of sub-propositional formula:', propositional_formula.first.first.root)
print('first child of sub-propositional formula:', propositional_formula.first.second.root)

root of propositional_formula: ~
first child of propositional formula: &
first child of sub-propositional formula: p
first child of sub-propositional formula: q


We can use recursion to convert an object of type PropositionalFormula into its corresponding string representation:

In [4]:
class PropositionalFormula:
    """
        Attributes:
            root: a constant, variable name, or operator
            first: the first operand of the root, if the root is a unary or binary operator.
            second:the second operand of the root, if the root is a binary operator.
    """
    def __init__(self, root: str, 
                 first: Self = None,
                 second: Self = None
                ):
        self.first = None
        self.second = None
        if is_variable(root) or is_constant(root):
            assert first is None and second is None
            self.root = root
        elif is_unary_operator(root):
            assert first is not None and second is None
            self.root, self.first = root, first
        else:
            assert is_binary_operator(root)
            assert first is not None and second is not None
            self.root, self.first, self.second = root, first, second
            
    def inorder(self):
        result = ''
        if self.root == '~':
            result += self.root
            result += self.first.inorder()
        else:
            if self.first:
                result += self.first.inorder()
            result += self.root
            if self.second:
                result += self.second.inorder()
        return result
        
    def __repr__(self):
        return self.inorder()

p = PropositionalFormula('p')
q = PropositionalFormula('q')
conjunction = PropositionalFormula('&', p, q)
negation = '~'
propositional_formula = PropositionalFormula('~', conjunction)
propositional_formula

~p&q

We can get the set of all of variables in a propositional formula using recursion as well:

In [5]:
class PropositionalFormula:
    """
        Attributes:
            root: a constant, variable name, or operator
            first: the first operand of the root, if the root is a unary or binary operator.
            second:the second operand of the root, if the root is a binary operator.
    """
    def __init__(self, root: str, 
                 first: Self = None,
                 second: Self = None
                ):
        self.first = None
        self.second = None
        if is_variable(root) or is_constant(root):
            assert first is None and second is None
            self.root = root
        elif is_unary_operator(root):
            assert first is not None and second is None
            self.root, self.first = root, first
        else:
            assert is_binary_operator(root)
            assert first is not None and second is not None
            self.root, self.first, self.second = root, first, second
            
    def get_propositional_formula(self) -> str:
        result = ''
        if self.root == '~':
            result += self.root
            result += self.first.get_propositional_formula()
        else:
            if self.first:
                result += self.first.get_propositional_formula()
            result += self.root
            if self.second:
                result += self.second.get_propositional_formula()
        return result
    
    def get_variables(self):
        leaves = self._get_variables(self)
        return set(leaves)

    def _get_variables(self, root):
        if not root: 
            return []
        if not root.first and not root.second: 
            return [root]
        leaves = self._get_variables(root.first) + self._get_variables(root.second)
        return leaves 

        
    def __repr__(self):
        return self.get_propositional_formula()

p = PropositionalFormula('p')
q = PropositionalFormula('q')
conjunction = PropositionalFormula('&', p, q)
negation = '~'
propositional_formula = PropositionalFormula('~', conjunction)


print('expression:', propositional_formula)
print('variables used in the expression:', end=' ')
print(propositional_formula.get_variables())

expression: ~p&q
variables used in the expression: {p, q}


We can use recursion to get thet set of all operators used in a propositional formula:

In [6]:
class PropositionalFormula:
    """
        Attributes:
            root: a constant, variable name, or operator
            first: the first operand of the root, if the root is a unary or binary operator.
            second:the second operand of the root, if the root is a binary operator.
    """
    def __init__(self, root: str, 
                 first: Self = None,
                 second: Self = None
                ):
        self.first = None
        self.second = None
        if is_variable(root) or is_constant(root):
            assert first is None and second is None
            self.root = root
        elif is_unary_operator(root):
            assert first is not None and second is None
            self.root, self.first = root, first
        else:
            assert is_binary_operator(root)
            assert first is not None and second is not None
            self.root, self.first, self.second = root, first, second
            
    def get_propositional_formula(self) -> str:
        result = ''
        if self.root == '~':
            result += self.root
            result += self.first.get_propositional_formula()
        else:
            if self.first:
                result += self.first.get_propositional_formula()
            result += self.root
            if self.second:
                result += self.second.get_propositional_formula()
        return result
    
    def get_variables(self, root):
        leaves = self._get_variables(root)
        return set(leaves)

    def _get_variables(self, root):
        if not root: 
            return []
        if not root.first and not root.second: 
            return [root]
        leaves = self._get_variables(root.first) + self._get_variables(root.second)
        return leaves 

    def get_operators(self) -> set:
        operators = self._get_operators(self)
        return set(operators)

    def _get_operators(self, root) -> list:
        if not root:
            return [] 
        if root.first or root.second: 
            return [root.root] + self._get_operators(root.first) + self._get_operators(root.second)
        return self._get_operators(root.first) + self._get_operators(root.second)
        
    def __repr__(self):
        return self.get_propositional_formula()

p = PropositionalFormula('p')
q = PropositionalFormula('q')
conjunction = PropositionalFormula('&', p, q)
negation = '~'
propositional_formula = PropositionalFormula('~', conjunction)


print('expression:', propositional_formula)
print('operators used in the expression:', end=' ')
print(propositional_formula.get_operators())

expression: ~p&q
operators used in the expression: {'&', '~'}
